### **1. Setup and load data**

##### **1.1 Install dependencies and data**

In [ ]:
import tensorflow as tf
import os #used for handling file operations

In [4]:
tf.config.list_physical_devices('GPU') #verify tensorflow is using the gpu

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# Avoid OOM errors. Setting GPU memory growth limit #### if this approach still doesn't resolve the memory consumption issue. we need to reduce size of the mini batches when training
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)